In [ ]:
#!pip install qiskit
#!pip install qiskit[visualization]

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer
import numpy as np

### Generate coin state

In [ ]:
n = 2
qc = QuantumCircuit(n)
qc.h(0)
for i in range(1,n):
    qc.cx(0,i)
    #qc.cx(i-1,i)
qc.draw('mpl')

Verify _coin_ state acts accordingly by running the quantum circuit in aer backend

In [ ]:
backend = Aer.get_backend('aer_simulator')
qc.save_statevector()
result = backend.run(qc).result()
counts = result.get_counts()
plot_histogram(counts)

### Generate Leader state

In [ ]:
def tensor_n_qudits(qudit, nr_times):
    partial_copy = qudit.copy()
    tensor_vec = qudit.tensor(partial_copy)
    for i in range(1,nr_times-1):
            tensor_vec = tensor_vec.tensor(partial_copy)
    return tensor_vec

In [ ]:
n_cube = pow(n,3)
norm = np.sqrt(n_cube)
nr_bits = np.floor(np.log2(n_cube)) + 1
dims_value = int(pow(2,nr_bits))

qudit = Statevector.from_int(1,dims=dims_value)
final_ket = (1 / norm) * tensor_n_qudits(qudit,n)

for i in range(2,n_cube+1):
    qudit = Statevector.from_int(i,dims=dims_value)
    final_ket += (1 / norm) * tensor_n_qudits(qudit,n)
display(final_ket.to_dict())

Verify _leader_ state acts accordingly by running the quantum circuit in aer backend

In [ ]:
circuit_nr_bits = n * nr_bits
another_qc = QuantumCircuit(circuit_nr_bits)
another_qc.initialize(final_ket.data)

backend = Aer.get_backend('aer_simulator')
another_qc.save_statevector()           # The final state of the simulator can be saved to the returned Result object by appending the save_state() instruction to a quantum circuit.
result = backend.run(another_qc).result()

counts = result.get_counts()
plot_histogram(counts)